In [1]:
import ccxt
import os
from dotenv import load_dotenv
import time
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo

In [2]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')


In [47]:
# Initialize the KuCoin exchange object
kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})
ORIGINAL_SYM_BIT = "BTC/USDT"
ORIGINAL_SYM_STR = "STRK/USDT"
symbol_btc = 'BTCUSDTM'
symbol_str = "STRKUSDTM"


# Helping Function

In [68]:
from kucoin_futures.client import Trade

def get_pos_size(sy):        
    client = Trade(key=kC_KEY, secret=kC_SECRET, passphrase=kC_PASSWORD)
    positions = client.get_position_details(symbol=sy)
    return positions['currentQty'], positions['isOpen']

def ask_bid_kucoi(symbol):
    order_book = kucoin.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]

def ask_bid_phemex(symbol):
    order_book = phemex.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]

# Example usage with a static symbol for Bitcoin
get_pos_size(sy=symbol_str)

(2, True)

# Daily SMA 20 Days

In [72]:
 #! FIND DAILY SMA PER 20 DAYS

def sma_20d():
    print("starting 20 Days ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=ORIGINAL_SYM_BIT, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()

    bid = ask_bid_kucoi(symbol=ORIGINAL_SYM_BIT)[1] 
    data_20days.loc[data_20days['SMA_20'] > bid, 'SIGNAL'] = 'SELL'
    data_20days.loc[data_20days['SMA_20'] < bid, 'SIGNAL'] = 'BUY'

    if data_20days['SIGNAL'].iloc[-1] == 'SELL':
        print("Opening as sell ...")
    else:
        print('Opening as Buy ...')
    
    return data_20days

sma_20d()


starting 20 Days ...
Opening as sell ...


,open,high,low,close,volume,date,SMA_20,SIGNAL
0,60302.00,62381.25,60231.99,61806.00,248.313128,2024-06-25,NaN,NaN
1,61790.00,62452.61,60708.07,60865.44,205.017520,2024-06-26,NaN,NaN
2,60863.27,62234.38,60678.05,61699.91,182.290141,2024-06-27,NaN,NaN
3,61690.00,62199.64,60136.85,60419.36,207.377027,2024-06-28,NaN,NaN
4,60420.72,61183.98,60377.87,60984.93,121.646566,2024-06-29,NaN,NaN
...,...,...,...,...,...,...,...,...
95,65769.95,66238.68,65418.90,65859.08,101.226382,2024-09-28,61526.7095,SELL
96,65858.00,66121.86,65438.00,65602.01,106.272116,2024-09-29,61958.7575,SELL
97,65602.38,65613.99,62910.01,63301.97,211.430687,2024-09-30,62240.6515,SELL
98,63325.00,64093.23,60341.34,60852.45,203.211621,2024-10-01,62416.1795,SELL


#  15 Minutes SMA

In [76]:
 #! Define the SMA for the 15 Minutes With Prices bp_1, bp_2, sp_1, sp_2

def sma_15min():
    print("starting sma 15 Minuts  ...")

    timeframe = '15m'
    num_bars = 100
    bars = kucoin.fetch_ohlcv(symbol=ORIGINAL_SYM_BIT, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_15m'] = data_15m['close'].rolling(window=20).mean()

    bid = ask_bid_kucoi(symbol=ORIGINAL_SYM_BIT)[1] 
        
    data_15m['bp_1'] = data_15m['SMA_15m'] * 1.001
    data_15m['bp_2'] = data_15m['SMA_15m'] * 0.997
    data_15m['sp_1'] = data_15m['SMA_15m'] * 0.999
    data_15m['sp_2'] = data_15m['SMA_15m'] * 1.003

    return data_15m

sma_15min()




starting sma 15 Minuts  ...


,open,high,low,close,volume,date,SMA_15m,bp_1,bp_2,sp_1,sp_2
0,61493.6,61493.7,61352.1,61483.3,17.865106,2024-10-02 05:45:00,NaN,NaN,NaN,NaN,NaN
1,61483.3,61569.0,61362.5,61554.0,10.224070,2024-10-02 06:00:00,NaN,NaN,NaN,NaN,NaN
2,61554.0,61608.1,61491.0,61589.9,5.588919,2024-10-02 06:15:00,NaN,NaN,NaN,NaN,NaN
3,61590.0,61766.5,61589.9,61754.5,12.332277,2024-10-02 06:30:00,NaN,NaN,NaN,NaN,NaN
4,61754.9,61775.4,61693.4,61693.4,12.119929,2024-10-02 06:45:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,61197.0,61221.0,61108.3,61213.5,17.331692,2024-10-03 05:30:00,61205.320,61266.525320,61021.704040,61144.114680,61388.935960
96,61212.1,61228.4,61119.8,61176.5,6.428896,2024-10-03 05:45:00,61214.795,61276.009795,61031.150615,61153.580205,61398.439385
97,61176.0,61278.4,61107.2,61247.9,11.024344,2024-10-03 06:00:00,61221.980,61283.201980,61038.314060,61160.758020,61405.645940
98,61247.9,61298.2,61241.9,61279.5,4.201154,2024-10-03 06:15:00,61227.745,61288.972745,61044.061765,61166.517255,61411.428235


# Bot

In [22]:
target = 25

def kill_switch():
    print("Starting the Kill switching ...")
    # TODO: Implement the Kill Switch
    pnl_c, _ ,side = pnl_close_kucoin() #! pnl_close, open_pos, Side-long,short
    size,open_p = get_pos_size(sy=symbol_str)

    print(f"open position: {open_p} side: {side} Size postion: {size}")
        
    
def pnl_close_kucoin():
    print("Starting PNL ...")
    
    # Example of different parameters

    poc_dic = kucoin.fetch_ticker(symbol='STRK/USDT')
    perc = poc_dic['percentage']
    diff = float(poc_dic['info']['changePrice'])
    current_price = poc_dic['last']
    entry_price = float(poc_dic['open']) 
    side = 'long' if (diff >= 0 and current_price >= entry_price) else 'short'
    
    print(f"Difference: {diff}, Current Price: {current_price}, Entry Price: {entry_price}, Side: {side}, Percentage: {perc}%")


    pnlclose = False
    in_pos = False
    if perc > 0:
        # * We are in a winning position (profit)
        if perc > target:
            # ! Closing the position as target is hit
            print("In Profit also Hit the Target ...")
            pnlclose = True
            kill_switch()
        else:
            print("In Profit but  not hit the target, so wait !!")
    else:
        in_pos = True
        print("We are not in Profit yet.")
    print("Finishing the PNL close ...")

    return pnlclose, in_pos,side

# print(pnl_close_phemex())
# pnl_close_kucoin()
kill_switch()


Starting the Kill switching ...
Starting PNL ...
Difference: 0.00487, Current Price: 0.4665, Entry Price: 0.46163, Side: long, Percentage: 1.05%
In Profit but  not hit the target, so wait !!
Finishing the PNL close ...
open position: True side: long Size postion: 2


In [11]:
def bot():
    data_15m = sma_15min()
    data_20d = sma_20d()
    ask,bid = ask_bid_kucoi(symbol=ORIGINAL_SYM_BIT)

    # * Define OPEN Order
    signal = data_20d.iloc[-1]['SIGNAL']
    
    open_size = 1
    
    in_pos = pnl_close()[1]


    if not in_pos:
        if signal == "BUY":
            print("Make order as BUY !!")
            bp_1 = data_15m.iloc[-1]['bp_1']
            bp_2 = data_15m.iloc[-1]['bp_2']
            print(f"bp_1: {bp_1} bp_2: {bp_2}")
            # TODO: Make Buy Order Here ...
        else:
            print("Make order as SELL !!")
            sp_1 = data_15m.iloc[-1]['sp_1']
            sp_2 = data_15m.iloc[-1]['sp_2']
            print(f"sp_1: {sp_1} sp_2: {sp_2}")
            # TODO: Make Sell Order Here ... 
    else:
        print("Already In Position, not making new Position")
bot()

2